# Description

Notebook for custom sklearn wrappers for exclusion threshold classifier.

# Imports

In [5]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import sys

from ipywidgets import interact_manual
from scipy.stats import norm
from scipy.special import logit
from sklearn.base import clone
from sklearn.datasets import make_regression
from sklearn.model_selection import KFold, GridSearchCV
from linearmodels.iv import IV2SLS

# Common issue with glibc, scikit (?), where the number of libraries loaded with static TLS is limited
# Can only load one of econml or rpy2...
# e.g. https://github.com/pytorch/pytorch/issues/2575#issuecomment-523657178
from econml.dml import CausalForestDML, LinearDML
from econml.grf import CausalForest

# user imports
sys.path.append("../")

# Don't import iv_power for now since it needs rpy2
from utils.pwr import rdd_power
from utils.sim import generate_IV_comply_indicator

# Sandbox wrappers

In [4]:
from sklearn.metrics import make_scorer

In [90]:
def iv_neff(y, y_pred, data=None):
    """
    Computes the effective sample size of the given data sample.
    
    Args:
        y (np.array): defined to match the function signature of a score, not used. 
        y_pred (np.array): binary indicator of whether a sample is included (1) or excluded (0)
        data (pd.DataFrame): the data frame containing T and Z
    
    """    
    data['sel'] = y_pred
    #print(data)
    sel_df = data[data['sel'] == 1].copy()
    
    comply_rate = sel_df[(sel_df['Z'] == 1)]['T'].mean() - sel_df[(sel_df['Z'] == 0)]['T'].mean()
    
    return sel_df.shape[0] * (comply_rate**2)

In [92]:
neff_scorer = make_scorer(iv_neff)

In [89]:
class ExcludeClassifier(CausalForest):
    """
    Wrapper class for CausalForest, with a threshold
    
    TODO modify so that y actually contains both Z and T, which can then be passed into scorer as "y"
    "y_pred" should still be allowed as a single score.
    """
    def __init__(self, threshold=0.5, n_estimators=100):
        self.threshold = threshold
        super().__init__(n_estimators=n_estimators)
    
    def fit(self, X, y, T=None):
        super().fit(X, T, y)
    
    def predict(self, X):
        preds = super().predict(X)
        
        return preds > self.threshold

## Make data

In [80]:
seed = 42
tau = 0.25
n_samples = 2000
n_feats = 5
regression_dict = dict(n_informative=5, noise=0, n_features=n_feats)

iv_df = generate_IV_comply_indicator(n_samples=n_samples, 
                                    tau=tau,
                                    prop_nt=0.30,
                                    prop_at=0.30,
                                    use_covars=True,
                                    regression_dict=regression_dict,
                                    seed=seed)

In [53]:
n_splits = 2

kfold = KFold(n_splits=n_splits,
             shuffle=True, random_state=42)

indices = []

# TODO be careful on what to split on
for train_idx, test_idx in kfold.split(iv_df): #, y=iv_df['Z']
    indices.append(test_idx)
    

s1_df = iv_df.iloc[indices[0]].copy()
s2_df = iv_df.iloc[indices[1]].copy()

In [54]:
feat_cols = ['feat_{}'.format(i) for i in range(n_feats)]

X = s1_df[feat_cols]
Y = s1_df['T'].values
T = s1_df['Z'].values

In [58]:
cforest = ExcludeClassifier()
cforest_cv = GridSearchCV(estimator=cforest,
                          param_grid={
                              "n_estimators": [100],
                              "threshold": [0.5, 0.7, 0.8]
                          },
                          scoring="accuracy", 
                          verbose=3)

In [59]:
%%time
cforest_cv.fit(X=X, y=Y, T=T)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV 1/5] END ...n_estimators=100, threshold=0.5;, score=0.560 total time=   0.4s
[CV 2/5] END ...n_estimators=100, threshold=0.5;, score=0.555 total time=   0.3s
[CV 3/5] END ...n_estimators=100, threshold=0.5;, score=0.575 total time=   0.3s
[CV 4/5] END ...n_estimators=100, threshold=0.5;, score=0.475 total time=   0.3s
[CV 5/5] END ...n_estimators=100, threshold=0.5;, score=0.575 total time=   0.3s
[CV 1/5] END ...n_estimators=100, threshold=0.7;, score=0.505 total time=   0.3s
[CV 2/5] END ...n_estimators=100, threshold=0.7;, score=0.490 total time=   0.3s
[CV 3/5] END ...n_estimators=100, threshold=0.7;, score=0.560 total time=   0.3s
[CV 4/5] END ...n_estimators=100, threshold=0.7;, score=0.450 total time=   0.3s
[CV 5/5] END ...n_estimators=100, threshold=0.7;, score=0.530 total time=   0.3s
[CV 1/5] END ...n_estimators=100, threshold=0.8;, score=0.480 total time=   0.3s
[CV 2/5] END ...n_estimators=100, threshold=0.8;,

GridSearchCV(estimator=ExcludeClassifier(),
             param_grid={'n_estimators': [100], 'threshold': [0.5, 0.7, 0.8]},
             scoring='accuracy', verbose=3)